In [34]:
import numpy as np
import pandas as pd
import pickle
# import faiss
from tqdm.notebook import tqdm

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.stats import entropy
from scipy import spatial

In [35]:
target_researchers_profile = pd.read_csv('./data/target_researchers_profile.csv')

In [36]:
id2name = {}
for i in range(len(target_researchers_profile)):
    id2name[target_researchers_profile.loc[i]['researcher_id']] = target_researchers_profile.loc[i]['PI']

In [37]:
# lda_pubid = pd.read_csv('./data/lda_pubid.csv')
pubid = pd.read_csv('./data/lda_pubid.csv')
pubid_year = pd.read_csv('./data/pubid_year.csv')
pub2re = pd.read_parquet('./data/target_abstracts_pub2researcher.parquet')
pub_cnt = pd.read_csv('./data/pub_count.csv')
foa_abstracts = pd.read_csv('./data/energy_grants_Dimensions_wAbstract_cleanest.csv')

In [29]:
potential_author_citing = pd.read_csv('./data/potential_author_citing.csv')
potential_author_referenced = pd.read_csv('./data/potential_author_referenced.csv')

potential_author_citing['citing'] = len(potential_author_citing) * [1]
potential_author_referenced['referenced'] = len(potential_author_referenced) * [1]

In [30]:
# j = 0
# all_vec = np.load(open('./data/lda/d100/embedding/d100_66_lda_part' + str(j) + '.npy', 'rb'))
# for j in tqdm(range(1, 62)):
#     all_vec = np.vstack([all_vec, np.load(open('./data/lda/d100/embedding/d100_66_lda_part' + str(j) + '.npy', 'rb'))])

# with open('./data/lda/d100/FOA_d100_66.npy', 'rb') as f:
#     vec_foa = np.load(f)

In [31]:
all_vec.shape

NameError: name 'all_vec' is not defined

In [30]:
vec_foa_df = pd.DataFrame(vec_foa)
vec_foa_df = vec_foa_df.set_index(pd.Index(foa_abstracts['Award_number'].values.tolist()))

In [31]:
grant_researcher = foa_abstracts[['Award_number', 'researcher_id']]

grant2re = grant_researcher.set_index('Award_number').T.to_dict('records')[0]

vec_df = pd.DataFrame(all_vec)
vec_df = vec_df.set_index(pd.Index(pubid['publication_id'].values.tolist()))

In [32]:
all_PI_idx = pd.read_csv('./data/all_PI_idx.csv')
sampled_id = all_PI_idx[all_PI_idx['category'] != 'potential']['researcher_id'].unique()

In [54]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []

for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = vec_df.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = vec_df.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-50:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()



In [55]:
pd.DataFrame(matching_dict)

,funded_name,funded_id,funded_npubs,funded_entropy,funded_FOA_similarity,matched_name,matched_id,matched_cosine_similarity,matched_npubs,matched_entropy,matched_FOA_similarity
0,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Emmanuel G Collins,ur.011471236145.91,1.0,54,4.605146,0.999971
1,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Joydeep Acharya,ur.010701015017.59,1.0,11,4.605147,0.999972
2,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Bren C Mochocki,ur.013765276155.27,1.0,11,4.605147,0.999975
3,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Lei Wu,ur.010322327104.06,1.0,12,4.605145,0.999969
4,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Yu Ding,ur.015716633211.16,1.0,46,4.605148,0.999973
...,...,...,...,...,...,...,...,...,...,...,...
5995,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Sean A Ramprashad,ur.010162606610.47,1.0,37,4.605143,0.999945
5996,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Joydeep Acharya,ur.010701015017.59,1.0,15,4.605142,0.999951
5997,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Kia Bazargan,ur.013344732015.91,1.0,38,4.605150,0.999957
5998,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Haomiao Huang,ur.012561141107.08,1.0,15,4.605145,0.999947


In [56]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_66.csv', index = False)
    
    
    

In [ ]:
coauthorship_weight = pd.read_csv('./data/pubid_coauthor_count.csv')
coauthorship_weight = coauthorship_weight.set_index(coauthorship_weight['publication_id']).drop(['publication_id'], axis = 1)
weighted_vec = coauthorship_weight.join(vec_df)
weighted_vec = weighted_vec.iloc[:, :].div(weighted_vec.coauthor_count, axis=0)
weighted_vec = weighted_vec.drop('coauthor_count', axis = 1)
coauthorship_weight['weight'] = coauthorship_weight['coauthor_count'].apply(lambda x: 1 / x)
coauthorship_weight = pd.read_csv('./data/pubid_coauthor_count.csv')

In [38]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = weighted_vec.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = weighted_vec.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-100:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()

In [40]:
pd.DataFrame(matching_dict)

,funded_name,funded_id,funded_npubs,funded_entropy,funded_FOA_similarity,matched_name,matched_id,matched_cosine_similarity,matched_npubs,matched_entropy,matched_FOA_similarity
0,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Jorge F Valenzuela,ur.012106654411.32,1.000000,22,4.605148,0.999975
1,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Chunting Mi,ur.011434721140.49,1.000000,13,4.605147,0.999975
2,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Daji Qiao,ur.07611240351.64,1.000000,52,4.605147,0.999974
3,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Girish Vinayak Chowdhary,ur.016663530513.65,1.000000,13,4.605146,0.999977
4,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Ilya V Kolmanovsky,ur.016131577413.61,1.000000,143,4.605149,0.999973
...,...,...,...,...,...,...,...,...,...,...,...
11995,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Jack D Reilly,ur.012423357433.35,0.999999,12,4.605139,0.999937
11996,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,John J Hasenbein,ur.013040047235.99,0.999999,21,4.605147,0.999941
11997,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Leonardo Bobadilla,ur.013073650035.26,0.999999,11,4.605143,0.999938
11998,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Nathan Wayne Fisher,ur.013125433747.65,0.999999,50,4.605142,0.999937


In [53]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_66_weighted.csv', index = False)
    
    
    

In [59]:
j = 0
all_vec = np.load(open('./data/lda/d100/embedding_default/d100_lda_part' + str(j) + '.npy', 'rb'))
for j in tqdm(range(1, 62)):
    all_vec = np.vstack([all_vec, np.load(open('./data/lda/d100/embedding_default/d100_lda_part' + str(j) + '.npy', 'rb'))])

In [60]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = vec_df.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = vec_df.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-50:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()



In [61]:
pd.DataFrame(matching_dict)

,funded_name,funded_id,funded_npubs,funded_entropy,funded_FOA_similarity,matched_name,matched_id,matched_cosine_similarity,matched_npubs,matched_entropy,matched_FOA_similarity
0,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Emmanuel G Collins,ur.011471236145.91,1.0,54,4.605146,0.999971
1,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Joydeep Acharya,ur.010701015017.59,1.0,11,4.605147,0.999972
2,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Bren C Mochocki,ur.013765276155.27,1.0,11,4.605147,0.999975
3,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Lei Wu,ur.010322327104.06,1.0,12,4.605145,0.999969
4,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Yu Ding,ur.015716633211.16,1.0,46,4.605148,0.999973
...,...,...,...,...,...,...,...,...,...,...,...
5995,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Sean A Ramprashad,ur.010162606610.47,1.0,37,4.605143,0.999945
5996,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Joydeep Acharya,ur.010701015017.59,1.0,15,4.605142,0.999951
5997,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Kia Bazargan,ur.013344732015.91,1.0,38,4.605150,0.999957
5998,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Haomiao Huang,ur.012561141107.08,1.0,15,4.605145,0.999947


In [62]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_default.csv', index = False)

In [63]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = weighted_vec.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = weighted_vec.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-100:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()

In [64]:
pd.DataFrame(matching_dict)

,funded_name,funded_id,funded_npubs,funded_entropy,funded_FOA_similarity,matched_name,matched_id,matched_cosine_similarity,matched_npubs,matched_entropy,matched_FOA_similarity
0,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Jorge F Valenzuela,ur.012106654411.32,1.000000,22,4.605148,0.999975
1,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Chunting Mi,ur.011434721140.49,1.000000,13,4.605147,0.999975
2,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Daji Qiao,ur.07611240351.64,1.000000,52,4.605147,0.999974
3,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Girish Vinayak Chowdhary,ur.016663530513.65,1.000000,13,4.605146,0.999977
4,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Ilya V Kolmanovsky,ur.016131577413.61,1.000000,143,4.605149,0.999973
...,...,...,...,...,...,...,...,...,...,...,...
11995,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Jack D Reilly,ur.012423357433.35,0.999999,12,4.605139,0.999937
11996,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,John J Hasenbein,ur.013040047235.99,0.999999,21,4.605147,0.999941
11997,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Leonardo Bobadilla,ur.013073650035.26,0.999999,11,4.605143,0.999938
11998,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Nathan Wayne Fisher,ur.013125433747.65,0.999999,50,4.605142,0.999937


In [65]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_default_weighted.csv', index = False)
    
    
    

# Matching for d68

In [32]:
with open('./data/lda/d68/d68_lda_part0.npy', 'rb') as f:
    vec_0 = np.load(f)
    
with open('./data/lda/d68/d68_lda_part1.npy', 'rb') as f:
    vec_1 = np.load(f)
    
with open('./data/lda/d68/d68_lda_part2.npy', 'rb') as f:
    vec_2 = np.load(f)
    
with open('./data/lda/d68/d68_lda_part3.npy', 'rb') as f:
    vec_3 = np.load(f)
    
with open('./data/lda/d68/d68_lda_part4.npy', 'rb') as f:
    vec_4 = np.load(f)
    
with open('./data/lda/d68/d68_lda_part5.npy', 'rb') as f:
    vec_5 = np.load(f)

with open('./data/lda/d68/d68_lda_part6.npy', 'rb') as f:
    vec_6 = np.load(f)

all_vec = np.vstack([vec_0, 
                     vec_1, 
                     vec_2,
                     vec_3,
                     vec_4,
                     vec_5,
                     vec_6])

del vec_0
del vec_1
del vec_2
del vec_3
del vec_4
del vec_5
del vec_6

with open('./data/lda/d68/FOA_d68.npy', 'rb') as f:
    vec_foa = np.load(f)

In [33]:
vec_foa_df = pd.DataFrame(vec_foa)
vec_foa_df = vec_foa_df.set_index(pd.Index(foa_abstracts['Award_number'].values.tolist()))

vec_df = pd.DataFrame(all_vec)
vec_df = vec_df.set_index(pd.Index(pubid['publication_id'].values.tolist()))

In [34]:
potential_author_bib = pd.read_csv('./data/potential_author_bib.csv')

In [35]:
potential_author_bib

,researcher_id,related_researcher_id
0,ur.0673371724.22,ur.07764700523.10
1,ur.01265637254.27,ur.0102676026.31
2,ur.01320646106.00,ur.01225774236.73
3,ur.01366372760.45,ur.0747341316.01
4,ur.0643236203.14,ur.0650755501.75
...,...,...
43750725,ur.015174017312.75,ur.013553162301.37
43750726,ur.01271424771.55,ur.01347532737.07
43750727,ur.01265450036.24,ur.01303476135.94
43750728,ur.01354244572.35,ur.013375123246.65


In [36]:
sampled_id = list(potential_author_bib['researcher_id'].unique())

In [37]:
len(sampled_id)

1965

In [ ]:
for y in [1, 3, 5, 10]:
    matching_dict = {}
    matching_dict['funded_name'] = []
    matching_dict['funded_id'] = []
    matching_dict['Award_number'] = []
    matching_dict['StartYear'] = []
    matching_dict['funded_entropy'] = []
    matching_dict['funded_FOA_similarity'] = []
    matching_dict['matched_name'] = []
    matching_dict['matched_id'] = []
    matching_dict['matched_cosine_similarity'] = []
    matching_dict['matched_entropy'] = []
    matching_dict['matched_FOA_similarity'] = []
    for target_rid in tqdm(sampled_id):
        target_name = id2name[target_rid]
        grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
        if len(grant_years) != 0:
            for grant_year in grant_years:
                Award_number = foa_abstracts[(foa_abstracts['researcher_id'] == target_rid) & (foa_abstracts['StartYear'] == grant_year)]['Award_number'].values[0]
                Award_LDA = vec_foa_df.loc[Award_number].values

                tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - y))]['publication_id'].values))]
                target_pubcount = len(tmp_pub)
                if target_pubcount >= 1:
                    tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

                    tmp_lda = vec_df.\
                        join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

                    target_lda = tmp_lda.mean(axis = 0).values
                    target_entropy = entropy(target_lda)

                    tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - y))]['publication_id'].values))]
                    tmp_other_pub = potential_author_bib[potential_author_bib['researcher_id'] == target_rid][['related_researcher_id']].\
                        rename({'related_researcher_id': 'researcher_id'}, axis = 1).\
                        merge(tmp_other_pub, how = 'inner', on = 'researcher_id')

                    other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
                    other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values

                    tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

                    other_lda = vec_df.\
                        reset_index().rename({'index': 'publication_id'}, axis = 1).\
                        merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                        drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

                    other_rid = other_lda.index.tolist()
                    other_entropy = entropy(other_lda.values, axis = 1)
                    other_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0]
#                     top_rid = [other_rid[idx] for idx in top_idx]
                    other_name = [id2name[rid] for rid in other_rid]

#                     top_entropy = [other_entropy[idx] for idx in top_idx]
#                     top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
#                     top_lda = other_lda.loc[top_rid]
                    other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), other_lda.values)
                    target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))

                    matching_dict['funded_name'] += [target_name] * len(other_name)
                    matching_dict['funded_id'] += [target_rid] * len(other_name)
                    matching_dict['Award_number'] += [Award_number] * len(other_name)
                    matching_dict['StartYear'] += [grant_year] * len(other_name)
                    matching_dict['funded_entropy'] += [target_entropy] * len(other_name)
                    matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(other_name)
                    matching_dict['matched_name'] += other_name
                    matching_dict['matched_id'] += other_rid
                    matching_dict['matched_cosine_similarity'] += other_dist.tolist()
                    matching_dict['matched_entropy'] += other_entropy.tolist()
                    matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()
    pd.DataFrame(matching_dict).to_csv('./data/matching_foa_researcher_1965_d68_' + str(y) + 'year.csv', index = False)

# Matching for d100

In [3]:
!ls ../sloan/full_abstracts/

all_PI_lda.csv
all_pub_lda.feather
average_lda_entropy.csv
centroid_matching_cosine_entropy.csv
centroid_matching_cosine_entropy_publication_abstract_counts.csv
centroid_matching_Kenneth_Gillingham.csv
corpus_full_abstracts_0.pickle
corpus_full_abstracts_1.pickle
corpus_full_abstracts_2.pickle
corpus_full_abstracts_3.pickle
corpus_full_abstracts_4.pickle
corpus_full_abstracts_5.pickle
corpus_full_abstracts_6.pickle
data_full_abstracts_0.pickle
data_full_abstracts_1.pickle
data_full_abstracts_2.pickle
data_full_abstracts_3.pickle
data_full_abstracts_4.pickle
data_full_abstracts_5.pickle
data_full_abstracts_6.pickle
FOA2_d68.npy
FOA_abstracts.pickle
FOA_d68.npy
FOA.npy
FOA_text.pickle
full_referencing_paper_lda.csv
lda_110d_fin_sampled_FOA_abstracts.npy
lda_110d_fin_sampled_PI_abstracts_0.npy
lda_110d_fin_sampled_PI_abstracts_1.npy
lda_110d_fin_sampled_PI_abstracts_2.npy
lda_110d_fin_sampled_PI_abstracts_3.npy
lda_110d_fin_sampled_PI_abstracts_4.npy
lda_110d_fin_sampled_PI_abstracts_5.np

In [39]:
with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_0.npy', 'rb') as f:
    vec_0 = np.load(f)
    
with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_1.npy', 'rb') as f:
    vec_1 = np.load(f)
    
with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_2.npy', 'rb') as f:
    vec_2 = np.load(f)
    
with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_3.npy', 'rb') as f:
    vec_3 = np.load(f)
    
with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_4.npy', 'rb') as f:
    vec_4 = np.load(f)
    
with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_5.npy', 'rb') as f:
    vec_5 = np.load(f)

with open('../sloan/full_abstracts/lda_110d_fin_sampled_PI_abstracts_6.npy', 'rb') as f:
    vec_6 = np.load(f)

all_vec = np.vstack([vec_0, 
                     vec_1, 
                     vec_2,
                     vec_3,
                     vec_4,
                     vec_5,
                     vec_6])

del vec_0
del vec_1
del vec_2
del vec_3
del vec_4
del vec_5
del vec_6

In [40]:
with open('../sloan/full_abstracts/lda_110d_fin_sampled_FOA_abstracts.npy', 'rb') as f:
    vec_foa = np.load(f)

In [9]:
vec_foa.shape

(2683, 110)

In [41]:
vec_foa_df = pd.DataFrame(vec_foa)
vec_foa_df = vec_foa_df.set_index(pd.Index(foa_abstracts['Award_number'].values.tolist()))

vec_df = pd.DataFrame(all_vec)
vec_df = vec_df.set_index(pd.Index(pubid['publication_id'].values.tolist()))

In [42]:
potential_author_bib = pd.read_csv('./data/potential_author_bib.csv')

In [43]:
sampled_id = list(potential_author_bib['researcher_id'].unique())

In [32]:
len(sampled_id)

1965

In [ ]:
for y in [1, 3, 5, 10]:
    matching_dict = {}
    matching_dict['funded_name'] = []
    matching_dict['funded_id'] = []
    matching_dict['Award_number'] = []
    matching_dict['StartYear'] = []
    matching_dict['funded_entropy'] = []
    matching_dict['funded_FOA_similarity'] = []
    matching_dict['matched_name'] = []
    matching_dict['matched_id'] = []
    matching_dict['matched_cosine_similarity'] = []
    matching_dict['matched_entropy'] = []
    matching_dict['matched_FOA_similarity'] = []
    for target_rid in tqdm(sampled_id):
        target_name = id2name[target_rid]
        grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
        if len(grant_years) != 0:
            for grant_year in grant_years:
                Award_number = foa_abstracts[(foa_abstracts['researcher_id'] == target_rid) & (foa_abstracts['StartYear'] == grant_year)]['Award_number'].values[0]
                Award_LDA = vec_foa_df.loc[Award_number].values

                tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - y))]['publication_id'].values))]
                target_pubcount = len(tmp_pub)
                if target_pubcount >= 1:
                    tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

                    tmp_lda = vec_df.\
                        join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

                    target_lda = tmp_lda.mean(axis = 0).values
                    target_entropy = entropy(target_lda)

                    tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - y))]['publication_id'].values))]
                    tmp_other_pub = potential_author_bib[potential_author_bib['researcher_id'] == target_rid][['related_researcher_id']].\
                        rename({'related_researcher_id': 'researcher_id'}, axis = 1).\
                        merge(tmp_other_pub, how = 'inner', on = 'researcher_id')

                    other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
                    other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values

                    tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

                    other_lda = vec_df.\
                        reset_index().rename({'index': 'publication_id'}, axis = 1).\
                        merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                        drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

                    other_rid = other_lda.index.tolist()
                    other_entropy = entropy(other_lda.values, axis = 1)
                    other_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0]
#                     top_rid = [other_rid[idx] for idx in top_idx]
                    other_name = [id2name[rid] for rid in other_rid]

#                     top_entropy = [other_entropy[idx] for idx in top_idx]
#                     top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
#                     top_lda = other_lda.loc[top_rid]
                    other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), other_lda.values)
                    target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))

                    matching_dict['funded_name'] += [target_name] * len(other_name)
                    matching_dict['funded_id'] += [target_rid] * len(other_name)
                    matching_dict['Award_number'] += [Award_number] * len(other_name)
                    matching_dict['StartYear'] += [grant_year] * len(other_name)
                    matching_dict['funded_entropy'] += [target_entropy] * len(other_name)
                    matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(other_name)
                    matching_dict['matched_name'] += other_name
                    matching_dict['matched_id'] += other_rid
                    matching_dict['matched_cosine_similarity'] += other_dist.tolist()
                    matching_dict['matched_entropy'] += other_entropy.tolist()
                    matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()
    pd.DataFrame(matching_dict).to_csv('./data/matching_foa_researcher_1965_d110_' + str(y) + 'year.csv', index = False)

In [2]:
year1 = pd.read_csv('./data/matching_foa_researcher_1965_d110_1year.csv')
year3 = pd.read_csv('./data/matching_foa_researcher_1965_d110_3year.csv')
year5 = pd.read_csv('./data/matching_foa_researcher_1965_d110_5year.csv')
year10 = pd.read_csv('./data/matching_foa_researcher_1965_d110_10year.csv')

In [29]:
year10[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'matched_name', 'matched_id', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'matched_cosine_similarity_10_year'}, axis = 1).\
    merge(year5[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'matched_name', 'matched_id', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'matched_cosine_similarity_5_year'}, axis = 1), 
          how = 'left', 
          on = ['funded_id', 'matched_id', 'funded_name', 'matched_name', 'Award_number', 'StartYear']).\
    merge(year3[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'matched_name', 'matched_id', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'matched_cosine_similarity_3_year'}, axis = 1), 
          how = 'left', 
          on = ['funded_id', 'matched_id', 'funded_name', 'matched_name', 'Award_number', 'StartYear']).\
    merge(year1[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'matched_name', 'matched_id', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'matched_cosine_similarity_1_year'}, axis = 1), 
          how = 'left', 
          on = ['funded_id', 'matched_id', 'funded_name', 'matched_name', 'Award_number', 'StartYear']).\
    fillna(0).\
    drop_duplicates().to_csv('./data/researcher_match_d110.csv', index = False)

In [23]:
year10[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_FOA_similarity']].rename({'funded_FOA_similarity': 'match_grant_prior10'}, axis = 1).\
    drop_duplicates().\
    merge(year5[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_FOA_similarity']].rename({'funded_FOA_similarity': 'match_grant_prior5'}, axis = 1), 
          how = 'left', 
          on = ['funded_id', 'funded_name', 'Award_number', 'StartYear']).\
    drop_duplicates().\
    merge(year3[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_FOA_similarity']].rename({'funded_FOA_similarity': 'match_grant_prior3'}, axis = 1), 
          how = 'left', 
          on = ['funded_id', 'funded_name', 'Award_number', 'StartYear']).\
    drop_duplicates().\
    merge(year1[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_FOA_similarity']].rename({'funded_FOA_similarity': 'match_grant_prior1'}, axis = 1), 
          how = 'left', 
          on = ['funded_id', 'funded_name', 'Award_number', 'StartYear']).\
    fillna(0).\
    groupby(['funded_id', 'funded_name', 'Award_number', 'StartYear']).head(1).reset_index(drop=True).\
    drop_duplicates().to_csv('./data/funded_foa_d110.csv', index = False)

In [24]:
pd.read_csv('./data/funded_foa_d110.csv')

,funded_name,funded_id,Award_number,StartYear,match_grant_prior10,match_grant_prior5,match_grant_prior3,match_grant_prior1
0,Sushil Adhikari,ur.0673371724.22,1333372,2013,0.810420,0.809715,0.746486,0.751190
1,Michael R Wasielewski,ur.01265637254.27,DE-SC0001059,2009,0.390888,0.387027,0.398047,0.380412
2,Hongjie Dai,ur.01320646106.00,DE-SC0008684,2012,0.822779,0.753456,0.700010,0.654905
3,Hongjie Dai,ur.01320646106.00,DE-SC0016165,2016,0.311693,0.454492,0.408361,0.360936
4,Vinayak P Dravid,ur.01366372760.45,DE-SC0014520,2015,0.385710,0.396406,0.426546,0.486655
...,...,...,...,...,...,...,...,...
2490,Ilenia Battiato,ur.01347533534.09,DE-SC0014227,2015,0.488474,0.487533,0.484998,0.662880
2491,Robert M Chin,ur.013765453431.50,1565893,2016,0.764131,0.787702,0.000000,0.000000
2492,David G Briggs,ur.010071136241.65,0855690,2009,0.172617,0.172617,0.171678,0.086933
2493,Simon W Evans,ur.010116351561.32,1336474,2013,0.921180,0.921180,0.921180,0.000000


In [19]:
year10[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'match_grant_prior10'}, axis = 1).\
    drop_duplicates().\
    merge(year5[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'match_grant_prior5'}, axis = 1), 
          how = 'left', 
          on = ['matched_id', 'matched_name', 'Award_number', 'StartYear']).\
    drop_duplicates().\
    merge(year3[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'match_grant_prior3'}, axis = 1), 
          how = 'left', 
          on = ['matched_id', 'matched_name', 'Award_number', 'StartYear']).\
    drop_duplicates().\
    merge(year1[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_cosine_similarity']].rename({'matched_cosine_similarity': 'match_grant_prior1'}, axis = 1), 
          how = 'left', 
          on = ['matched_id', 'matched_name', 'Award_number', 'StartYear']).\
    fillna(0).\
    groupby(['matched_id', 'matched_name', 'Award_number', 'StartYear']).head(1).reset_index(drop=True).\
    drop_duplicates().to_csv('./data/matched_foa_d110.csv', index = False)

In [20]:
pd.read_csv('./data/matched_foa_d110.csv')

,matched_name,matched_id,Award_number,StartYear,match_grant_prior10,match_grant_prior5,match_grant_prior3,match_grant_prior1
0,Keri B Cantrell,ur.01000251401.10,1333372,2013,0.605088,0.556996,0.624612,0.0
1,Xiao Zhang,ur.01001256402.55,1333372,2013,0.166550,0.200805,0.000000,0.0
2,Dieter Von Deak,ur.010017061415.34,1333372,2013,0.377869,0.358658,0.000000,0.0
3,Mark Crocker,ur.010033624216.66,1333372,2013,0.663993,0.662242,0.612086,0.0
4,Craig Stuart Frear,ur.01004173504.77,1333372,2013,0.489565,0.680023,0.000000,0.0
...,...,...,...,...,...,...,...,...
2625843,Nibir K Dhar,ur.07647536161.16,1338820,2013,0.884652,0.736249,0.621755,0.0
2625844,Humberto Terrones,ur.0765110175.62,1338820,2013,0.343799,0.420127,0.515032,0.0
2625845,Brian T Mayers,ur.0774320522.01,1338820,2013,0.269378,0.000000,0.000000,0.0
2625846,Alan H Chin,ur.0777206324.11,1338820,2013,0.862083,0.000000,0.000000,0.0


In [25]:
year10[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_entropy']].rename({'funded_entropy': 'entropy_prior10'}, axis = 1).\
    drop_duplicates().\
    merge(year5[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_entropy']].rename({'funded_entropy': 'entropy_prior5'}, axis = 1), on = ['funded_name', 'funded_id', 'Award_number', 'StartYear'], how = 'left').\
    drop_duplicates().\
    merge(year3[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_entropy']].rename({'funded_entropy': 'entropy_prior3'}, axis = 1), on = ['funded_name', 'funded_id', 'Award_number', 'StartYear'], how = 'left').\
    drop_duplicates().\
    merge(year1[['funded_name', 'funded_id', 'Award_number', 'StartYear', 'funded_entropy']].rename({'funded_entropy': 'entropy_prior1'}, axis = 1), on = ['funded_name', 'funded_id', 'Award_number', 'StartYear'], how = 'left').\
    drop_duplicates().fillna(0).\
    groupby(['funded_name', 'funded_id', 'Award_number', 'StartYear']).head(1).reset_index(drop=True).\
    rename({'StartYear': 'start_year', 'funded_id': 'researcher_id', 'funded_name': 'PI', 'Award_number': 'award_number'}, axis = 1).\
    to_csv('./data/funded_entropy_110d.csv', index = False)

In [27]:
pd.read_csv('./data/funded_entropy_110d.csv')

,PI,researcher_id,award_number,start_year,entropy_prior10,entropy_prior5,entropy_prior3,entropy_prior1
0,Sushil Adhikari,ur.0673371724.22,1333372,2013,2.192296,2.192424,2.207624,1.804721
1,Michael R Wasielewski,ur.01265637254.27,DE-SC0001059,2009,2.070260,2.110263,2.172102,1.992356
2,Hongjie Dai,ur.01320646106.00,DE-SC0008684,2012,2.727759,2.867878,2.793532,2.832177
3,Hongjie Dai,ur.01320646106.00,DE-SC0016165,2016,2.939858,2.975691,2.961624,2.705173
4,Vinayak P Dravid,ur.01366372760.45,DE-SC0014520,2015,2.758356,2.722470,2.727229,2.548351
...,...,...,...,...,...,...,...,...
2490,Ilenia Battiato,ur.01347533534.09,DE-SC0014227,2015,2.088196,2.121703,2.221504,2.008460
2491,Robert M Chin,ur.013765453431.50,1565893,2016,1.339400,1.188311,0.000000,0.000000
2492,David G Briggs,ur.010071136241.65,0855690,2009,1.924120,1.924120,1.822729,1.749322
2493,Simon W Evans,ur.010116351561.32,1336474,2013,1.197378,1.197378,1.197378,0.000000


In [26]:
year10[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_entropy']].rename({'matched_entropy': 'entropy_prior10'}, axis = 1).\
    drop_duplicates().\
    merge(year5[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_entropy']].rename({'matched_entropy': 'entropy_prior5'}, axis = 1), on = ['matched_name', 'matched_id', 'Award_number', 'StartYear'], how = 'left').\
    drop_duplicates().\
    merge(year3[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_entropy']].rename({'matched_entropy': 'entropy_prior3'}, axis = 1), on = ['matched_name', 'matched_id', 'Award_number', 'StartYear'], how = 'left').\
    drop_duplicates().\
    merge(year1[['matched_name', 'matched_id', 'Award_number', 'StartYear', 'matched_entropy']].rename({'matched_entropy': 'entropy_prior1'}, axis = 1), on = ['matched_name', 'matched_id', 'Award_number', 'StartYear'], how = 'left').\
    drop_duplicates().fillna(0).\
    reset_index(drop = True).\
    groupby(['matched_name', 'matched_id', 'Award_number', 'StartYear']).head(1).reset_index(drop=True).\
    rename({'StartYear': 'start_year', 'matched_id': 'researcher_id', 'matched_name': 'PI', 'Award_number': 'award_number'}, axis = 1).\
    to_csv('./data/matched_entropy_110d.csv', index = False)

In [28]:
pd.read_csv('./data/matched_entropy_110d.csv')

,PI,researcher_id,award_number,start_year,entropy_prior10,entropy_prior5,entropy_prior3,entropy_prior1
0,Keri B Cantrell,ur.01000251401.10,1333372,2013,2.548194,2.565068,2.491785,0.0
1,Xiao Zhang,ur.01001256402.55,1333372,2013,2.309348,2.550619,0.000000,0.0
2,Dieter Von Deak,ur.010017061415.34,1333372,2013,1.472043,1.472043,0.000000,0.0
3,Mark Crocker,ur.010033624216.66,1333372,2013,2.100155,2.170532,2.219374,0.0
4,Craig Stuart Frear,ur.01004173504.77,1333372,2013,2.484731,2.625644,0.000000,0.0
...,...,...,...,...,...,...,...,...
2625843,Nibir K Dhar,ur.07647536161.16,1338820,2013,1.781136,1.913306,1.907838,0.0
2625844,Humberto Terrones,ur.0765110175.62,1338820,2013,2.502339,2.451408,2.440166,0.0
2625845,Brian T Mayers,ur.0774320522.01,1338820,2013,2.493231,0.000000,0.000000,0.0
2625846,Alan H Chin,ur.0777206324.11,1338820,2013,1.651062,0.000000,0.000000,0.0


# Lift 10 pub restriction

In [ ]:
for y in [1, 3, 5, 10]:
    matching_dict = {}
    matching_dict['funded_name'] = []
    matching_dict['funded_id'] = []
    matching_dict['Award_number'] = []
    matching_dict['StartYear'] = []
    matching_dict['funded_entropy'] = []
    matching_dict['funded_FOA_similarity'] = []
    matching_dict['matched_name'] = []
    matching_dict['matched_id'] = []
    matching_dict['matched_cosine_similarity'] = []
    matching_dict['matched_entropy'] = []
    matching_dict['matched_FOA_similarity'] = []
    for target_rid in tqdm(sampled_id):
        target_name = id2name[target_rid]
        grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
        if len(grant_years) != 0:
            for grant_year in grant_years:
                Award_number = foa_abstracts[(foa_abstracts['researcher_id'] == target_rid) & (foa_abstracts['StartYear'] == grant_year)]['Award_number'].values[0]
                Award_LDA = vec_foa_df.loc[Award_number].values

                tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - y))]['publication_id'].values))]
                target_pubcount = len(tmp_pub)
                if target_pubcount >= 1:
                    tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

                    tmp_lda = vec_df.\
                        join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

                    target_lda = tmp_lda.mean(axis = 0).values
                    target_entropy = entropy(target_lda)

                    tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - y))]['publication_id'].values))]
                    tmp_other_pub = potential_author_bib[potential_author_bib['researcher_id'] == target_rid][['related_researcher_id']].\
                        rename({'related_researcher_id': 'researcher_id'}, axis = 1).\
                        merge(tmp_other_pub, how = 'inner', on = 'researcher_id')

                    other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
                    other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 1]['researcher_id'].values

                    tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

                    other_lda = vec_df.\
                        reset_index().rename({'index': 'publication_id'}, axis = 1).\
                        merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                        drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

                    other_rid = other_lda.index.tolist()
                    other_entropy = entropy(other_lda.values, axis = 1)
                    other_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0]
#                     top_rid = [other_rid[idx] for idx in top_idx]
                    other_name = [id2name[rid] for rid in other_rid]

#                     top_entropy = [other_entropy[idx] for idx in top_idx]
#                     top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
#                     top_lda = other_lda.loc[top_rid]
                    other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), other_lda.values)
                    target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))

                    matching_dict['funded_name'] += [target_name] * len(other_name)
                    matching_dict['funded_id'] += [target_rid] * len(other_name)
                    matching_dict['Award_number'] += [Award_number] * len(other_name)
                    matching_dict['StartYear'] += [grant_year] * len(other_name)
                    matching_dict['funded_entropy'] += [target_entropy] * len(other_name)
                    matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(other_name)
                    matching_dict['matched_name'] += other_name
                    matching_dict['matched_id'] += other_rid
                    matching_dict['matched_cosine_similarity'] += other_dist.tolist()
                    matching_dict['matched_entropy'] += other_entropy.tolist()
                    matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()
    pd.DataFrame(matching_dict).to_csv('./data/matching_foa_researcher_1965_d110_' + str(y) + '_no_restrict_year.csv', index = False)